In [24]:
import folium
import pandas as pd
from folium.plugins import HeatMapWithTime
from datetime import datetime, timedelta

def create_taxi_heatmap(df):
    
    # Group data by pickup zone and 30-minute intervals
    grouped_data = df.groupby([
        'pickup_zone', 
        df['30_minute_interval']
    ]).size().reset_index(name='count')
    
    grouped_data = pd.merge(
        grouped_data, 
        df[['pickup_zone', 'pickup_lat', 'pickup_lon']].drop_duplicates(),
        on='pickup_zone', 
        how='left'
    )
    
    all_intervals = sorted(df['30_minute_interval'].unique())
    interval_data = []
    for interval in all_intervals:
        interval_group = grouped_data[grouped_data['30_minute_interval'] == interval]
        locations = [[row['pickup_lat'], row['pickup_lon'], row['count']] 
                     for _, row in interval_group.iterrows()]
        interval_data.append(locations)
    
    nyc_map = folium.Map(
        location=[40.7528, -74.0060],
        zoom_start=11.5,
        tiles='cartodbpositron'
    )
    
    time_labels = [pd.Timestamp(interval).strftime('%H:%M') for interval in all_intervals]
    
    HeatMapWithTime(
        interval_data,
        index=time_labels,
        auto_play=True,
        max_opacity=0.7,
        radius=45,        # Reduced radius for finer detail
        min_opacity=0.05,  # Lower min_opacity for better gradients
        gradient={0.01: 'blue', 0.65: 'lime', 0.8: 'yellow', 1: 'red'},  # Added color gradient
        use_local_extrema=True,
        display_index=True,
    ).add_to(nyc_map)
    
    title_html = '''
        <div style="position: fixed; 
                    top: 10px; 
                    left: 50px; 
                    z-index:9999; 
                    background-color: white;
                    border: 2px solid black;
                    border-radius: 5px;
                    padding: 10px;
                    font-size: 16px;
                    font-weight: bold;">
            NYC Taxi Pickups by 30-Minute Intervals
        </div>
    '''
    nyc_map.get_root().html.add_child(folium.Element(title_html))
    
    return nyc_map

df = pd.read_csv("../../data/taxi_data.csv")
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['30_minute_interval'] = df['tpep_pickup_datetime'].dt.floor('30min')
map_with_slider = create_taxi_heatmap(df)
map_with_slider.save('collaborative.html')


In [13]:
grouped_data = df.groupby([
        'pickup_zone', 
        df['10_minute_interval']
    ]).size().reset_index(name='count')
    
pd.merge(
        grouped_data, 
        df[['pickup_zone', 'pickup_lat', 'pickup_lon']].drop_duplicates(),
        on='pickup_zone', 
        how='left'
    ).describe()

,10_minute_interval,count,pickup_lat,pickup_lon
count,82757,82757.000000,82757.000000,82757.000000
mean,2021-01-16 18:36:19.305678336,2.044588,40.843408,-77.434235
min,2009-01-01 00:00:00,1.000000,-37.933340,-123.214451
25%,2021-01-09 06:10:00,1.000000,40.735863,-74.002742
50%,2021-01-16 16:40:00,1.000000,40.758013,-73.980064
75%,2021-01-24 12:10:00,3.000000,40.787011,-73.960208
max,2021-01-31 23:50:00,19.000000,55.902400,151.271066
std,NaN,1.597022,1.861465,11.934914
